# 6950

In [1]:
# !pip install PyRuSH
# !pip install pyConTextNLP
# !pip install textblob
# !pip install radnlp

## Rules

In [2]:
sentence_rules='KB/rush_rules.tsv'
#target_rules='KB/social_kb.yml'
#context_rules='KB/general_modifiers.yml'
#feature_inference_rule='KB/featurer_inferences.csv'
#document_inference_rule='KB/doc_inferences.csv'


In [3]:
with open('KB/social_target_rules_042118.yml','r') as f:   # 'KB/social_target_rules_040618.yml'
    target_rules = f.read()
with open('KB/social_modifiers_2018.yml','r') as f:   # KB/lexical_kb_05042016.yml , KB/general_modifiers_2018.yml
    context_rules = f.read()
with open('KB/featurer_inferences.csv','r') as f:     
    feature_inference_rule = f.read()
with open('KB/doc_inferences.csv','r') as f:  
    document_inference_rule = f.read()

## NLP pipeline

In [4]:
from pynlp_pipe import Mypipe

In [5]:
myPipe=Mypipe(sentence_rules, target_rules, context_rules, feature_inference_rule, document_inference_rule)

## Read filenames

In [6]:
import os
#path ="test/2017/combine"                    # 2017 96+439
path = "test/500_1/corpus"                    # 500 notes
#path = "test/156_156/soc"                    # combine positive

files = os.listdir(path)
len(files)


120

## Read documents and apply pipeline

In [7]:
import chardet

results=dict()  # dictionary will contain document names as keys and a document-level classification as values.
context_doc_res=[]

# Read txt files
doc_texts = []
#note_count = 0                       # count the number of text notes want to process ***
for i in files[:]:
    if ".txt" in i:
        #note_count = note_count + 1  #
        #if note_count > 20:          # count the number of text notes want to process ***
        #    break                    #
        with open(os.path.join(path,i), 'rb') as f:
            doc_txt = chardet.detect(f.read()) 
        #print(i)
        #print(result["encoding"])        
        
        with open(os.path.join(path,i),encoding=doc_txt["encoding"]) as f:
            doc_text = f.read()  # or readline if the file is large     
            doc_text=doc_text.replace('\n', ' ')
                        
            doc_class, context_doc, annotations, relations = myPipe.process(doc_text)
            if len(annotations) != 0 and doc_class == 'no_isolation':
                print (i)
            results[i] = doc_class
            context_doc_res.append(context_doc)

## Print classification results

In [8]:
res_num=0
for j in results:
    print(j, " : ", results[j])
    res_num=res_num+1
    if res_num > 5:
        break


09696212_199472695_469775619.txt  :  isolation_doc
09910266_203543374_585135519.txt  :  isolation_doc
09914912_202033473_535253049.txt  :  isolation_doc
10882066_216071107_853913484.txt  :  no_isolation
10882066_216071107_859257422.txt  :  no_isolation
10882066_216071107_863455238.txt  :  no_isolation


## Validation

In [9]:
posPath ="test/500_1/corpusp"      # 500 notes
#posPath ="test/2017/isolation"     # 2017 96+439
#posPath ="test/156_156/soc_pos"    # combine positive
negPath = "test/500_1/corpusn"     # 500 notes
#negPath = "test/2017/noisolation"  # 2017 96+439
#negPath ="test/156_156/soc_neg"    # combine positive

posLab = "isolation_doc"
negLab = "no_isolation"

In [10]:
from pynlp_valid import Validnote

In [11]:
validnote = Validnote()
std_doc = validnote.readstd(posPath, negPath, posLab, negLab)

In [12]:
precision, recall, f1 = validnote.validation(results, std_doc, posLab, negLab)
print("*"*20)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

fn: 11016888_205116311_615981390.txt
fp: 14144257_216657674_865911455.txt
fp: 14260145_211642830_760314646.txt
fp: 14498364_203097610_559618901.txt
fn: 15157993_211465548_756717735.txt
fn: 20427474_205942535_631639605.txt
fn: 20530486_199785789_477860675.txt
fn: 20841417_202481581_544378727.txt
fn: 20878801_211192075_752731208.txt
fn: 20960880_208010073_683357038.txt
fn: 21020299_210268292_734687814.txt
fn: 21030541_210790602_762893652.txt
fn: 21057691_211766501_765109852.txt
------------------------------
	Reference 	 Total
------------------------------
System 	 50 	 3 	 53
System 	 10 	 57 	 67
------------------------------
Total 	 60 	 60
------------------------------
********************
Precision:  0.9433962264150944
Recall:  0.8333333333333334
F1:  0.8849557522123894


In [13]:
with open(os.path.join(path,"14498364_203097610_559618901.txt")) as f:
    doc_text = f.read()      
    doc_text=doc_text.replace('\n', ' ')                        
    doc_class, context_doc, annotations, relations = myPipe.process(doc_text)

In [14]:
annotations

,markup_id,vis_category,start,end,txt,type
0,T0,Target,68,79,felt lonely,isolation


In [15]:
relations

,relation_id,type,arg1_cate,arg1_id,arg2_cate,arg2_id
